In [42]:
import numpy as np

In [6]:
import logging
import time
import pandas as pd
from colorama import Fore

from deeppavlov import build_model, configs


logging.basicConfig(level=logging.ERROR)


def timed_inference(model, question: str, context: str, print_q_v: bool = True):
    start_time = time.perf_counter()
    prediction = model([context], [question])[0][0]
    predict_time = time.perf_counter() - start_time

    if print_q_v:
        print(Fore.BLUE + f"Question: {question}")
        print(Fore.YELLOW + f"Variants: {context}")
    print(Fore.GREEN + f"[{predict_time:.3f}] Predicted: `{prediction}`")


In [3]:
model = build_model(configs.squad.squad)

[nltk_data] Downloading package punkt to /Users/ruagmn9/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ruagmn9/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /Users/ruagmn9/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /Users/ruagmn9/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2021-11-12 13:32:44.414 INFO in 'deeppavlov.models.preprocessors.squad_preprocessor'['squad_preprocessor'] at line 310: SquadVocabEmbedder: loading saved tokens vocab from /Users/ruagmn9/.deeppavlov/models/squad_model/emb/vocab_embedder.pckl
INFO:deeppavlov.models.preprocessors.squad_preprocessor:SquadVocabEmbedder: loading saved tokens vocab from /Users/ruagmn9/.deeppavlov/models/squad_model/emb/vocab_embedder.pckl


In [4]:
timed_inference(model, "Раз два три четыре пять вышел зайчик что делать?", "Поплавать. Погулять. Покурить. Попрыгать.")

Question: Раз два три четыре пять вышел зайчик что делать?
Variants: Поплавать. Погулять. Покурить. Попрыгать.
[1.667] Predicted: `Погулять`


In [70]:
data = pd.read_csv('/Users/ruagmn9/PycharmProjects/raif-bootcamp-2021/data/boot_camp_train.csv')

In [71]:
data = data.rename(columns = {'Вопрос': 'question', 'Правильный ответ': 'target'})

In [72]:
data = data.dropna(subset=['question']).fillna('Пусто')

In [73]:
data['context'] = data[['1', '2', '3', '4']].agg('. '.join, axis=1)

In [74]:
%%time

data['predict'] = data.apply(lambda x: model([x['context']], [x['question']])[0][0], axis=1)

CPU times: user 6min 59s, sys: 2min 8s, total: 9min 7s
Wall time: 1min 46s


In [75]:
data

,Unnamed: 0,question,1,2,3,4,target,context,predict
0,0,"Как же, согласно поговорке, все-таки там, где ...",Тепло,Сытно,Хорошо,Весело,3.0,Тепло. Сытно. Хорошо. Весело,Хорошо
1,4,Какую спортивную игру начинают не с подачи?,Теннис,Волейбол,Баскетбол,Бадминтон,3.0,Теннис. Волейбол. Баскетбол. Бадминтон,Волейбол
2,9,На чьи стихи написана песня «День победы?»,Добронравов,Матусовский,Харитонов,Ошанин,3.0,Добронравов. Матусовский. Харитонов. Ошанин,Матусовский
3,10,Кто регулярно тушит свет на канале НТВ?,Лев Новожёнов,Леонид Парфёнов,Елена Ханга,Анатолий Чубайс,1.0,Лев Новожёнов. Леонид Парфёнов. Елена Ханга. А...,Лев
4,11,"По мнению пословицы, плох солдат, который не м...",Маршалом,Генералом,Полковником,Дембелем,2.0,Маршалом. Генералом. Полковником. Дембелем,Дембелем
...,...,...,...,...,...,...,...,...,...
8995,9266,Как называется хищное растение?,«Венерина мухоловка»,«Марсова пчелоедка»,«Сатурнова клоподавка»,«Юпитеров слепнежуй»,1.0,«Венерина мухоловка». «Марсова пчелоедка». «Са...,мухоловка
8996,9267,Что бывает у шапки?,«Полянка»,«Тропинка»,«Опушка»,«Чащоба»,3.0,«Полянка». «Тропинка». «Опушка». «Чащоба»,Полянка». «Тропинка
8997,9268,Что не использовалось для разогрева утюгов раз...,Скипидар,Уголь,Спирт,Электричество,1.0,Скипидар. Уголь. Спирт. Электричество,Скипидар
8998,9269,В каком турнире капитан команды обычно не выхо...,Кубок Дэвиса по теннису,Кубок Стэнли по хоккею,Кубок Европы по баскетболу,Кубок мира по борьбе,1.0,Кубок Дэвиса по теннису. Кубок Стэнли по хокке...,Дэвиса


In [76]:
def select_answer_based_on_predict(predict_str,
                                  series):
    for index, answer in enumerate(series, 1 ):
        if predict_str in answer:
            return index
    return 1

In [77]:
data['predict_index'] = data.apply(lambda x: select_answer_based_on_predict(x['predict'],
                                                                            x[['1','2', '3', '4']]),
                                                                            axis=1)

In [78]:
from sklearn.metrics import accuracy_score

In [81]:
accuracy_score(data['target'], data['predict_index'])

0.2538059784420491

In [83]:
accuracy_score(data['target'], np.ones(data.shape[0]))

0.24180464496055118